## 1.환경 설정

In [6]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## 2. MCP 서버 연결 (sse)
- MCP 서버 수행 후 연결

In [9]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import ainvoke_graph, astream_graph
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(
    model="claude-3-haiku-20240307",
    max_tokens=4096,
    temperature=0
)

async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    answer = await astream_graph(agent, {"messages": "오늘 서울 시간을 알려줘"})
    print(answer)

[StructuredTool(name='get_current_time', description='\n    Get current time for the specified timezone.\n    \n    This function returns the current time for the specified timezone.\n    If no timezone is specified, the default timezone is "Asia/Seoul".\n\n    Args:\n        timezone (str): The timezone to get the current time for. Default is "Asia/Seoul".\n\n    Returns:\n        str: A string containing the current time for the specified timezone.\n    ', args_schema={'properties': {'timezone': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': 'Asia/Seoul', 'title': 'Timezone'}}, 'title': 'get_current_timeArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x16a71f2e0>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Current time in Asia/Seoul: 2025-05-08 15:39:41
🔄 Node: agent 🔄
- - - - - 

## 3. MCP 서버 연결 (stdio)

In [10]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_anthropic import ChatAnthropic

# Anthropic 모델 생성
model = ChatAnthropic(
    model_name="claude-3-haiku-20240307",
    max_tokens=4096,
    temperature=0
)

# StdIO 서버 파라미터 설정
# - command: Python 인터프리터 경로
# - args: 실행할 MCP 서버 스크립트 경로
server_params = StdioServerParameters(
    command="./.venv/bin/python",
    args=["01_mcp_server_local.py"],
)

# StdIO 클라이언트를 사용하여 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()
        
        # 도구 로드
        tools = await load_mcp_tools(session)
        print(tools)
        
        # 에이전트 생성
        agent = create_react_agent(model, tools)
        
        # 도구 호출
        await astream_graph(agent, {"messages": "오늘 서울의 날씨는 어때?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x16a71f060>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
The weather in 서울 is sunny.
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - -